In [1]:
!pip install -U tensorflow_hub
!pip install tf-nightly-gpu

Requirement already up-to-date: tensorflow_hub in /home/badc0ded/anaconda3/envs/colab/lib/python3.6/site-packages (0.5.0)


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
import tensorflow as tf
tf.enable_eager_execution()
import tensorflow_hub as hub
from tensorflow.keras import layers

W0728 11:55:19.108352 139998109325120 module_wrapper.py:136] From /home/badc0ded/anaconda3/envs/colab/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.1
session = tf.Session(config=config)
import tensorflow.keras.backend as K
K.set_session(session=session)

In [4]:
import matplotlib.pylab as plt
import numpy as np
import PIL.Image as Image
import time
import os

In [5]:
os.environ['DATA_PATH'] = '/home/badc0ded/notebooks/data'
#os.environ['DATA_PATH'] = '/notebooks/data/datasets/pipistrel/Hackathon/SingleFrame_ObjectProposalClassification'

In [6]:
DATA_PATH = os.environ.get('DATA_PATH')

IMAGE_SHAPE = (224, 224)

In [7]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.batch_acc.append(logs['precision'])
    self.batch_acc.append(logs['recall'])
    self.model.reset_metrics()


def read_data(data_root):
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255)
  image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SHAPE)
  return image_data


def plot_predicted_batch(image_batch, label_batch, predicted_batch):
  predicted_id = np.argmax(predicted_batch, axis=-1)
  predicted_label_batch = class_names[predicted_id]
  label_id = np.argmax(label_batch, axis=-1)
  plt.figure(figsize=(10, 9))
  plt.subplots_adjust(hspace=0.5)
  for n in range(30):
    plt.subplot(6, 5, n + 1)
    plt.imshow(image_batch[n])
    color = "green" if predicted_id[n] == label_id[n] else "red"
    plt.title(predicted_label_batch[n].title(), color=color)
    plt.axis('off')
  _ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

In [8]:
train_data_root = os.path.join(DATA_PATH, "train")
train_image_data = read_data(train_data_root)
train_image_batch, train_label_batch = train_image_data[0]

Found 11390 images belonging to 2 classes.


In [9]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=IMAGE_SHAPE + (3,))
feature_extractor_layer(train_image_batch)
feature_extractor_layer.trainable = False

model = tf.keras.Sequential([
  #layers.Dropout(0.2, input_shape=(224,224,3)),
  feature_extractor_layer,
  #layers.Dense((train_image_data.num_classes +  1280) / 2, activation = 'relu'),
  layers.Dense(train_image_data.num_classes, activation='softmax')
])

model.summary()

In [10]:
load_path = '/home/badc0ded/notebooks/saved_models/best_10_02'
reloaded = tf.keras.experimental.load_from_saved_model(load_path,
                                                       custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])


W0728 11:55:23.680005 139998109325120 deprecation.py:323] From <ipython-input-10-b27862bb1eaf>:3: load_from_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
W0728 11:55:27.454114 139998109325120 module_wrapper.py:136] From /home/badc0ded/anaconda3/envs/colab/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



In [11]:
def eval_model_on_data(model, data_root_short_name):
    data_root = os.path.join(DATA_PATH, data_root_short_name)
    image_data = read_data(data_root)
    loss, acc = model.evaluate(image_data)
    print('Accuracy: {}'.format(acc))
    return loss, acc, image_data

In [12]:
%%time
_, test_acc, test_image_data = eval_model_on_data(reloaded, "test")

Found 4614 images belonging to 2 classes.
145/145 [==============================] - 88s 604ms/step - loss: 0.0048 - acc: 0.9989
Accuracy: 0.9989163279533386
CPU times: user 4min 20s, sys: 43.1 s, total: 5min 3s
Wall time: 1min 28s


In [13]:
_, test_boats_acc, test_boats_image_data = eval_model_on_data(reloaded, "test_boats")

Found 482 images belonging to 2 classes.
16/16 [==============================] - 9s 579ms/step - loss: 0.0183 - acc: 0.9979
Accuracy: 0.9979252815246582


In [14]:
_, train_boats_acc, train_boats_image_data = eval_model_on_data(reloaded, "train_boats")

Found 1011 images belonging to 2 classes.
32/32 [==============================] - 20s 637ms/step - loss: 0.0141 - acc: 0.9980
Accuracy: 0.9980217814445496


In [15]:
print('Missed {} out of {} boats in test.zip'.format(
    int(test_boats_image_data.samples * (1 - test_boats_acc)),
    test_boats_image_data.samples))
print('Missed {} out of {} boats in train.zip'.format(
    int(train_boats_image_data.samples * (1 - train_boats_acc)),
    train_boats_image_data.samples))

Missed 1 out of 482 boats in test.zip
Missed 1 out of 1011 boats in train.zip
